In [1]:
import pandas as pd

## Выбор кейса 

В данной работе я ставлю перед собой следущую бизнес-цель: решить задачу **бинарной классификации вакансий на настоящие и мошеннические**. В данной задаче классы сильно несбалансированны, так как мошеннических вакансий обычно существенно меньше, чем настоящих.

Данная цель имеет право на жизнь, так как соотвествующий классификатор можно встроить в антифрод-систему сервисов с объявлениями о работе, что в конечном счете может принести дополнительную прибыль данному сервису.

## Сбор данных

### Датасеты для исходной задачи

Для своей задачи я нашел два подходящих датасета:
1. [Real / Fake Job Posting Prediction](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction) -- датасет с 18 тыс. вакансий, из которых около 800 являются фейковыми
2. [Fake Job Postings](https://www.kaggle.com/datasets/srisaisuhassanisetty/fake-job-postings) -- датасет из 10 тыс. фейковых вакансий

Оба датасета имеют лицензию [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/), поэтому их можно использовать для обучающих, исследовательских и коммерческих целей, не спрашивая разрешения у авторов.

Загрузим датасеты и прочитаем их

In [ ]:
# !mkdir data/real_fake_postings/ && cd data/real_fake_postings/ && kaggle datasets download shivamb/real-or-fake-fake-jobposting-prediction && unzip real-or-fake-fake-jobposting-prediction.zip 
# !mkdir data/fake_postings/ && cd data/fake_postings/ && kaggle datasets download srisaisuhassanisetty/fake-job-postings && unzip fake-job-postings.zip 

In [ ]:
real_fake_df = pd.read_csv("data/real_fake_postings/fake_job_postings.csv")
fake_df = pd.read_csv("data/fake_postings/Fake Postings.csv")

In [9]:
real_fake_df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [10]:
fake_df.head()

,title,description,requirements,company_profile,location,salary_range,employment_type,industry,benefits,fraudulent
0,Mental health nurse,Arm drive court sure vote. Earn $5000/week! Im...,"Basic knowledge in live, no degree required. F...",Rivera and Sons - Established 2022.,West Jeffrey,$55016-$100476,Internship,IT,Free meals,1
1,Conference centre manager,Government whom its bed go tax tree black. Ear...,"Basic knowledge in seek, no degree required. F...","Davidson, Jones and Gomez - Established 2003.",Lake Meredithberg,$53438-$93138,Part-Time,Finance,Flexible hours,1
2,"Engineer, land",I member discuss follow way there nation. Earn...,"Basic knowledge in worker, no degree required....",Allen Ltd - Established 1998.,Lake Cathybury,$45584-$105229,Part-Time,IT,Free travel,1
3,Forest/woodland manager,House across wait approach face. Earn $5000/we...,"Basic knowledge in example, no degree required...",Forbes Ltd - Established 1990.,South Matthewstad,$66188-$139621,Full-Time,Education,Free travel,1
4,"Production designer, theatre/television/film",Case best environmental full finally leader me...,"Basic knowledge in smile, no degree required. ...","Jennings, Martin and Sanchez - Established 1975.",East Rhondafurt,$32183-$115012,Temporary,Retail,Flexible hours,1


### Weak Supervision

#### Датасет из смежной задачи
Из смежной задачи, которая заключается в предсказании зарплаты по данным вакансии, я нашёл датасет [LinkedIn Job Postings (2023 - 2024)](https://www.kaggle.com/datasets/arshkon/linkedin-job-postings/data) с более 100 тыс. вакансиями из LinkedIn. В этом датасете нет информации о том, является ли заданное объявляение фейком или нет, но я воспользуюсь Weak Supervision, чтобы доразметить его.

Датасет имеет лицензию [CC BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/), поэтому его можно использовать для исследовательских и коммерческих целей, но выпуская наш продукт под той же лицензией с указанием ссылки на исходный датасет.
